In [1]:
import torch
import pandas as pd
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import torch.nn.functional as F
from sklearn.model_selection import ParameterGrid
from sklearn.neural_network import MLPClassifier
from transformers import AutoModelForSequenceClassification, AutoModel, AutoTokenizer
from transformers import RobertaForSequenceClassification
from transformers import RobertaTokenizer
from torch.utils.data import DataLoader, RandomSampler, TensorDataset

C:\Users\15527\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Evaluation (F-1)

In [2]:
from sklearn.metrics import classification_report
from typing import Dict
import numpy as np
class SharedTaskConstants:
    """
    Use these constants to interface with the data, not with the id2label used
    inside the Huggingface models!!
    """
    targets = ['validity', 'novelty']
    validity_label_mapping = {
        -1: "not-valid",
        0: "not-valid",  # can be excluded since test set does not contain these
        1: "valid",
    }

    novelty_label_mapping = {
        -1: "not-novel",
        0: "not-novel",  # can be excluded since test set does not contain these
        1: "novel",
    }

    validity_id2label = {v: k for k, v in validity_label_mapping.items()}
    novelty_id2label = {v: k for k, v in novelty_label_mapping.items()}

    local_str_mapping = {
        'novel': 1,
        'not-novel': 0,
        'valid': 1,
        'not-valid': 0
    }

    @staticmethod
    def val_nov_metric(is_validity: np.ndarray, should_validity: np.ndarray, is_novelty: np.ndarray,
                       should_novelty: np.ndarray) -> Dict[str, float]:
        ret = dict()

        ret_base_help = {
            "true_positive_validity": np.sum(np.where(
                np.all(np.stack([is_validity >= .5, should_validity >= .5]), axis=0),
                1, 0)),
            "true_positive_novelty": np.sum(np.where(
                np.all(np.stack([is_novelty >= .5, should_novelty >= .5]), axis=0),
                1, 0)),
            "true_positive_valid_novel": np.sum(np.where(
                np.all(np.stack([is_validity >= .5, is_novelty >= .5,
                                 should_validity >= .5, should_novelty >= .5]), axis=0),
                1, 0)),
            "true_positive_nonvalid_novel": np.sum(np.where(
                np.all(np.stack([is_validity < .5, is_novelty >= .5,
                                 should_validity < .5, should_novelty >= .5]), axis=0),
                1, 0)),
            "true_positive_valid_nonnovel": np.sum(np.where(
                np.all(np.stack([is_validity >= .5, is_novelty < .5,
                                 should_validity >= .5, should_novelty < .5]), axis=0),
                1, 0)),
            "true_positive_nonvalid_nonnovel": np.sum(np.where(
                np.all(np.stack([is_validity < .5, is_novelty < .5,
                                 should_validity < .5, should_novelty < .5]), axis=0),
                1, 0)),
            "classified_positive_validity": np.sum(np.where(is_validity >= .5, 1, 0)),
            "classified_positive_novelty": np.sum(np.where(is_novelty >= .5, 1, 0)),
            "classified_positive_valid_novel": np.sum(np.where(
                np.all(np.stack([is_validity >= .5, is_novelty >= .5]), axis=0),
                1, 0)),
            "classified_positive_nonvalid_novel": np.sum(np.where(
                np.all(np.stack([is_validity < .5, is_novelty >= .5]), axis=0),
                1, 0)),
            "classified_positive_valid_nonnovel": np.sum(np.where(
                np.all(np.stack([is_validity >= .5, is_novelty < .5]), axis=0),
                1, 0)),
            "classified_positive_nonvalid_nonnovel": np.sum(np.where(
                np.all(np.stack([is_validity < .5, is_novelty < .5]), axis=0),
                1, 0)),
            "indeed_positive_validity": np.sum(np.where(should_validity >= .5, 1, 0)),
            "indeed_positive_novelty": np.sum(np.where(should_novelty >= .5, 1, 0)),
            "indeed_positive_valid_novel": np.sum(np.where(
                np.all(np.stack([should_validity >= .5, should_novelty >= .5]), axis=0),
                1, 0)),
            "indeed_positive_nonvalid_novel": np.sum(np.where(
                np.all(np.stack([should_validity < .5, should_novelty >= .5]), axis=0),
                1, 0)),
            "indeed_positive_valid_nonnovel": np.sum(np.where(
                np.all(np.stack([should_validity >= .5, should_novelty < .5]), axis=0),
                1, 0)),
            "indeed_positive_nonvalid_nonnovel": np.sum(np.where(
                np.all(np.stack([should_validity < .5, should_novelty < .5]), axis=0),
                1, 0)),
        }

        ret_help = {
            "precision_validity": ret_base_help["true_positive_validity"] /
                                  max(1, ret_base_help["classified_positive_validity"]),
            "precision_novelty": ret_base_help["true_positive_novelty"] /
                                 max(1, ret_base_help["classified_positive_novelty"]),
            "recall_validity": ret_base_help["true_positive_validity"] /
                               max(1, ret_base_help["indeed_positive_validity"]),
            "recall_novelty": ret_base_help["true_positive_novelty"] /
                              max(1, ret_base_help["indeed_positive_novelty"]),
            "precision_valid_novel": ret_base_help["true_positive_valid_novel"] /
                                     max(1, ret_base_help["classified_positive_valid_novel"]),
            "precision_valid_nonnovel": ret_base_help["true_positive_valid_nonnovel"] /
                                        max(1, ret_base_help["classified_positive_valid_nonnovel"]),
            "precision_nonvalid_novel": ret_base_help["true_positive_nonvalid_novel"] /
                                        max(1, ret_base_help["classified_positive_nonvalid_novel"]),
            "precision_nonvalid_nonnovel": ret_base_help["true_positive_nonvalid_nonnovel"] /
                                           max(1, ret_base_help["classified_positive_nonvalid_nonnovel"]),
            "recall_valid_novel": ret_base_help["true_positive_valid_novel"] /
                                  max(1, ret_base_help["indeed_positive_valid_novel"]),
            "recall_valid_nonnovel": ret_base_help["true_positive_valid_nonnovel"] /
                                     max(1, ret_base_help["indeed_positive_valid_nonnovel"]),
            "recall_nonvalid_novel": ret_base_help["true_positive_nonvalid_novel"] /
                                     max(1, ret_base_help["indeed_positive_nonvalid_novel"]),
            "recall_nonvalid_nonnovel": ret_base_help["true_positive_nonvalid_nonnovel"] /
                                        max(1, ret_base_help["indeed_positive_nonvalid_nonnovel"])
        }

        ret.update({
            "f1_validity": 2 * ret_help["precision_validity"] * ret_help["recall_validity"] / max(1e-4, ret_help[
                "precision_validity"] + ret_help["recall_validity"]),
            "f1_novelty": 2 * ret_help["precision_novelty"] * ret_help["recall_novelty"] / max(1e-4, ret_help[
                "precision_novelty"] + ret_help["recall_novelty"]),
            "f1_valid_novel": 2 * ret_help["precision_valid_novel"] * ret_help["recall_valid_novel"] / max(1e-4,
                                                                                                           ret_help[
                                                                                                               "precision_valid_novel"] +
                                                                                                           ret_help[
                                                                                                               "recall_valid_novel"]),
            "f1_valid_nonnovel": 2 * ret_help["precision_valid_nonnovel"] * ret_help["recall_valid_nonnovel"] / max(
                1e-4, ret_help["precision_valid_nonnovel"] + ret_help["recall_valid_nonnovel"]),
            "f1_nonvalid_novel": 2 * ret_help["precision_nonvalid_novel"] * ret_help["recall_nonvalid_novel"] / max(
                1e-4, ret_help["precision_nonvalid_novel"] + ret_help["recall_nonvalid_novel"]),
            "f1_nonvalid_nonnovel": 2 * ret_help["precision_nonvalid_nonnovel"] * ret_help[
                "recall_nonvalid_nonnovel"] / max(1e-4, ret_help["precision_nonvalid_nonnovel"] + ret_help[
                "recall_nonvalid_nonnovel"])
        })

        ret.update({
            "f1_macro": (ret["f1_valid_novel"] + ret["f1_valid_nonnovel"] + ret["f1_nonvalid_novel"] + ret[
                "f1_nonvalid_nonnovel"]) / 4
        })

        return ret

In [3]:
def print_results(baseline_name: str, y_true: dict, y_pred: dict):
    print(f"==== {baseline_name} ====")
    print("Validity")
    results_validity = classification_report(
        y_true['validity'],
        y_pred['validity'],
        target_names=['not-valid', 'valid'],
        labels=[0, 1],
        zero_division=0
    )
    print(results_validity)

    print("Novelty")
    results_novelty = classification_report(
        y_true['novelty'],
        y_pred['novelty'],
        target_names=['not-novel', 'novel'],
        labels=[0, 1],
        zero_division=0
    )
    print(results_novelty)

    print("Combined (organization eval)")
    res = SharedTaskConstants.val_nov_metric(
        np.array(y_pred['validity']),
        np.array(y_true['validity']),
        np.array(y_pred['novelty']),
        np.array(y_true['novelty']),
    )
    print(res['f1_macro'].round(4))

### Load Train and Test

In [4]:
def str_to_list(text):
    '''
    Return tensor string into list
    '''
    # clean string
    clean_str = text.replace('tensor(', '').replace(')', '').strip()
    # convert to list
    tensor = eval(clean_str, {"torch": torch, "__builtins__": {}})
    return tensor

def process_covariate_data(df):
    '''
    Expanding all tensors in a single cell
    Make confidence into ordinal variables
    '''
    # convert str to tensor (list)
    SBERT_premise = df.SBERT_premise.apply(lambda x: str_to_list(x))
    SBERT_conclusion = df.SBERT_conclusion.apply(lambda x: str_to_list(x))

    # expand the list into individual entries
    df_expand1 = SBERT_premise.apply(pd.Series)
    df_expand2 = SBERT_conclusion.apply(pd.Series)

    # assign a meaningful name
    df_expand1.columns = ['pre_emb{}'.format(i+1) for i in range(df_expand1.shape[1])]
    df_expand2.columns = ['con_emb{}'.format(i+1) for i in range(df_expand2.shape[1])]

    # put everything together
    df_final = pd.concat([df.drop(['SBERT_premise', "SBERT_conclusion"], axis=1), df_expand1, df_expand2], axis=1)
    return df_final

def preprocess_input(x, y):
    '''
    return DataLoader for later input into the model
    '''
    # pd.dataframe to array
    x = np.array(x, dtype=np.float64)
    y = np.array(y, dtype=np.float64)
    # transform y for nn model
    # assume a value of 0 is not valid/novel
    y[y == -1] = 0
    # Transform the data
    transformation_dict = {
    (1, 1): [1, 0, 0, 0],
    (1, 0): [0, 1, 0, 0],
    (0, 1): [0, 0, 1, 0],
    (0, 0): [0, 0, 0, 1],
    }
    
    y = np.array([transformation_dict[tuple(row)] for row in y])
    # array to tensor
    x_torch = torch.tensor(x)
    y_torch = torch.tensor(y)
    data = TensorDataset(x_torch, y_torch)

    batch_size = 10
    loader = DataLoader(data, batch_size=batch_size, shuffle=True)
    return loader

In [5]:
train = pd.read_csv("../Data/TaskA_train_neural_kg.csv", index_col=False)
test = pd.read_pickle("../Data/TaskA_test_neural_kg.pkl")

In [6]:
X_train = train.loc[:,["SBERT_cosine_sim",'Irrelevancy', 'AveDistance']]
y_train = train.loc[:,["Validity",'Novelty']]
y_train = y_train.replace(-1, 0)

X_test = test.loc[:,["SBERT_cosine_sim",'Irrelevancy', 'AveDistance']]
y_test = test.loc[:,["Validity",'Novelty']]
y_test = y_test.replace(-1, 0)

### NN under Sklearn MLP

In [7]:
clf_valid = MLPClassifier(solver='adam', hidden_layer_sizes=(2, 2), alpha=0.1, random_state=259, max_iter=1000)
clf_valid.fit(X_train, y_train.Validity)

clf_novel = MLPClassifier(solver='adam', hidden_layer_sizes=(2, 2), alpha=0.1, random_state=259, max_iter=1000)
clf_novel.fit(X_train, y_train.Novelty)

pred = {}
pred["validity"] = clf_valid.predict(np.array(X_test))
pred["novelty"] = clf_novel.predict(np.array(X_test))
true_y = {"validity": list(y_test.Validity), "novelty": list(y_test.Novelty)}

print_results("MLP", pred, true_y)

==== MLP ====
Validity
              precision    recall  f1-score   support

   not-valid       0.31      0.62      0.41       102
       valid       0.88      0.66      0.75       418

    accuracy                           0.65       520
   macro avg       0.59      0.64      0.58       520
weighted avg       0.76      0.65      0.68       520

Novelty
              precision    recall  f1-score   support

   not-novel       1.00      0.57      0.72       520
       novel       0.00      0.00      0.00         0

    accuracy                           0.57       520
   macro avg       0.50      0.28      0.36       520
weighted avg       1.00      0.57      0.72       520

Combined (organization eval)
0.2236


C:\Users\15527\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
C:\Users\15527\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


### ROBERT Classfier

#### Prepare model inputs

In [8]:
train_input = train.topic + " [SEP] " + train.Premise + " [SEP] " + train.Conclusion
test_input = test.topic + " [SEP] " + test.Premise + " [SEP] " + test.Conclusion

In [9]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
def get_embeddings(model, text):
    # Tokenize sentences
    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)
    # Perform pooling. In this case, max pooling.
    sentence_embeddings = mean_pooling(model_output, encoded_input["attention_mask"])
    return sentence_embeddings.cpu().squeeze(0)

# Get embedding
# tokenizer = AutoTokenizer.from_pretrained("usc-isi/sbert-roberta-large-anli-mnli-snli")
# model = AutoModel.from_pretrained("usc-isi/sbert-roberta-large-anli-mnli-snli")
# print("Get SBERT embeddings for a datapoint")
# total_embedding = train_input.apply(lambda x: get_embeddings(model, x))
# Embedding_Matrix = torch.stack(total_embedding.tolist())

In [20]:
# Embedding from the RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
encoded_X_train = tokenizer(train_input.tolist(), padding=True, truncation=True, return_tensors='pt', max_length=512)
encoded_X_test = tokenizer(test_input.tolist(), padding=True, truncation=True, return_tensors='pt', max_length=512)

In [21]:
# Convert dataset to tensors and create a DataLoader
y_train_valid = torch.tensor([label for label in y_train.Validity])
y_test_valid = torch.tensor([label for label in y_test.Validity])

dataset_train = TensorDataset(encoded_X_train['input_ids'].squeeze(), encoded_X_train['attention_mask'].squeeze(), y_train_valid)
trainloader = DataLoader(dataset_train, sampler=RandomSampler(dataset_train), batch_size=25)

dataset_test = TensorDataset(encoded_X_test['input_ids'].squeeze(), encoded_X_test['attention_mask'].squeeze(), y_test_valid)
testloader = DataLoader(dataset_test, sampler=RandomSampler(dataset_test), batch_size=25)

In [15]:
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels = 2)

# Setup training
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()
for epoch in range(2):
    for batch in trainloader:
        optimizer.zero_grad()
        inputs, att_mask, labels = batch
        labels = labels.long().view(-1)
        outputs = model(input_ids = inputs, attention_mask = att_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"epoch: {epoch}, loss: {loss}")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


epoch: 0, loss: 0.8204604983329773
epoch: 1, loss: 0.8237802982330322


In [25]:
for batch in trainloader:
    inputs, att_mask, labels = batch
print(inputs.shape)
print(att_mask.shape)
print(labels.shape)


torch.Size([25, 128])
torch.Size([25, 128])
torch.Size([25])


#### Experiment

In [18]:
# small sample prediction (all the same?)
model.eval()
input_ids = tokenizer(["Hello, my dog is cute [ SEP ] dog is cute", "terrible day [ SEP ] day is bad", "Food is delicious [ SEP ] Food taste bad"], padding=True, truncation=True, return_tensors='pt', max_length=512)["input_ids"]  # Batch size 1
mask = tokenizer(["Hello, my dog is cute [ SEP ] dog is cute", "terrible day [ SEP ] day is bad", "Food is delicious [ SEP ] Food taste bad"], padding=True, truncation=True, return_tensors='pt', max_length=512)["attention_mask"]  # Batch size 1

truelabels = torch.tensor([1,1,0]).unsqueeze(0)  # Batch size 1
outputs = model(input_ids, mask)
print(outputs.logits)

tensor([[0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087]], grad_fn=<AddmmBackward0>)


### Making Prediction

In [19]:

model.eval()
with torch.no_grad():
    for batch in testloader:
        inputs, att_mask, labels = batch
        outputs = model(input_ids = inputs, attention_mask = att_mask)
        predictions = torch.argmax(outputs.logits, dim=1)
        prob_valid = F.softmax(outputs.logits, dim=1)
    print(prob_valid)

tensor([[0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087]])
tensor([[0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.2578, 0.0087],
        [0.